# Import libraries

In [1]:
import sys
import os
import time
import numpy as np
from snAPI.Main import *
import Motor_f as mot

ModuleNotFoundError: No module named 'zaber_motion'

# Functions

In [ ]:
def focusing(handlez,sn,z0,z1,step):
    sn = snAPI(libType=LibType.HH)
    sn.getDevice()
    sn.initDevice(MeasMode.T2)
    sn.device.setSyncDiv(1)
    sn.timeTrace.setNumBins(10000)
    #sn.timeTrace.setHistorySize(10)  
    sn.timeTrace.measure(100000, False, False)
    count_arr=np.array([])
    z_arr=np.arange(z0,z1,step,wait)
    for zi in z_arr:
        mot.move_abs_z(zdrive,zi)
        time.sleep()
        counts, _ = sn.timeTrace.getData() 
        count_arr=np.append(count_arr,counts)
    sn.stopMeasure()
    return count_arr,z_arr  

In [ ]:
def XY_scan(handlex,handley,x0,xf,Nx,y0,yf,Ny,wait,folder):# positions in micrometers
    #Create the scanning pattern (it is important to be ordered to minimize displacements)
    xs=np.linspace(x0,xf,Nx)
    ys=np.linspace(y0,yf,Ny)
    XY=np.meshgrid(xs,ys)
    Order=np.zeros_like(XY[0])
    for i in range(0,np.shape(XY[0])[0]):
        for j in range(0,np.shape(XY[0])[1]):
            if j%2==0:
                Order[i,j]=i+Ny*j
            else:
                Order[i,j]=Nx-i-1+Ny*j
    xx,yy,o=XY[0].flatten(),XY[1].flatten(),Order.flatten()
    xsorted=[x for _, x in sorted(zip(o,xx))]
    ysorted=[x for _, x in sorted(zip(o, yy))]
    pairs=np.vstack((xsorted,ysorted)).T
    # The points of the scan are saved at pairs (ordered)
    for point in pairs:
        mot.move_abs_xy(handlex,point[0]/1000) #move functions work in milimeters
        mot.move_abs_xy(handley,point[1]/1000)
        time.sleep(wait)
        sn.setPTUFilePath("./Data/{}/PTU_{:.2f}_{:.2f}.ptu".format(folder,point[0],point[1]))
        sn.histogram.setRefChannel(0)
        sn.histogram.setBinWidth(1000)
        sn.histogram.measure(1000,savePTU=True)
    return True

# Connections

## Connect motors

### Piezos (for scanning)

In [ ]:
xdrive=mot.connectX()
ydrive=mot.connectY()

---- Connecting PDXC motor ----
Devices=[['01403418', '0403&6015&THORLABS PDXC 1.51&THORLABS&COM11&COM'], ['01404934', '0403&6015&THORLABS PDXC 1.51&THORLABS&COM10&COM']]
connect  01403418
Found PDX1 stage
-- Stage settings --
-  closed loop mode  -
set loop to closed loop
-  Manual Mode  -
set trigger mode to manual mode
set PositionCalibration home
get CalibrationIsCompleted: done
set TargetSpeed to 10
---- Connecting PDXC motor ----
Devices=[['01403418', '0403&6015&THORLABS PDXC 1.51&THORLABS&COM11&COM'], ['01404934', '0403&6015&THORLABS PDXC 1.51&THORLABS&COM10&COM']]
connect  01404934
Found PDX1 stage
-- Stage settings --
-  closed loop mode  -
set loop to closed loop
-  Manual Mode  -
set trigger mode to manual mode
set PositionCalibration home
get CalibrationIsCompleted: done
set TargetSpeed to 10
Found Device 1 SN: 61516 (T-NA08A25) -> Connection 1 (Binary Serial port: COM5) devices
-Homed-


### Z axis (For focusing)

In [4]:
zdrive=mot.connectZ()

NameError: name 'mot' is not defined

## Connect HydraHarp

In [4]:
sn = snAPI(libType=LibType.HH)
sn.getDeviceIDs()
print('Available devices are:'+str(sn.deviceIDs))

True

In [ ]:
sn.getDevice()
sn.getDeviceConfig()
print(sn.deviceConfig)

In [ ]:
sn.initDevice(MeasMode.T3)

# Measurements

In [5]:
XY_scan(xdrive,ydrive,0,1,5,0,1,5,1,'Trial')

True

In [12]:
sn.setPTUFilePath("./PTU.ptu")
sn.histogram.setRefChannel(0)
sn.histogram.setBinWidth(10000)
sn.histogram.measure(10000,savePTU=True)
data, bins = sn.histogram.getData()